In [9]:
from whoosh.index import create_in
from whoosh.fields import *
import os
import re

In [10]:
schema = Schema(title=TEXT(stored=True), path=ID(stored=True), content=TEXT)
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")
ix = create_in("indexdir", schema)

In [11]:
import numpy as np
import datetime as dt

wd = 'downloaded_transcripts'

In [12]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [13]:
test = open(wd+"/AI_courses/A Nested Attention Neural Hybrid Model for Grammatical Error Correction _ ACL 2017-SI9cTcyNVew.en.vtt", "r", encoding='utf-8').read()
vtt0 = test.split('\n\n')

def cleanAutogenSubtitles(subtitles):
    line = [item for item in subtitles[2].split('\n') if item is not ' ']
    try:
        if(len(line[0].split(' --> ')[1]) > 12):
            cleaned = []
            for x in subtitles: 
                line = [item for item in x.split('\n') if item is not ' ']
                line[0] = line[0][:29]
                if(len(line) == 2 and len(cleanhtml(line[1])) == len(line[1])):
                    cleaned.append(line)
            subtitles = cleaned
    except:
        return subtitles
    
    return subtitles
            
# cleanAutogenSubtitles(vtt0)


In [14]:
next(os.walk('.'))[1]

subdir_ls = [(x[0],x[2]) for x in os.walk(wd) if len(x[2])>0] #only get directories with files

#part 0: Create dict of files
file_name_ls = []
file_content_ls = []
for i, subdir in enumerate(subdir_ls):
    print(str(i+1)+' of '+str(len(subdir_ls)), end='\r')
    subdir_path, subdir_files = subdir[0], subdir[1]
    file_name_ls += subdir_files
    for f in subdir_files:
        vtt0 = open(subdir_path+"/"+f, "r", encoding='utf-8').read()
        vtt1 = cleanAutogenSubtitles(vtt0.split('\n\n'))
        if vtt1 != ['']:
            vtt2 = []
            for x in vtt1:
                if(len(x)>0):
                    try:
                        tmp = x.split('\n')
                        timepoint = tmp[0].split(' --> ')
                        timepoint = timepoint[0] + '\t' + timepoint[1]   #convert to string
                        label = f + '\t' + timepoint
                        vtt2.append((label, ' '.join(tmp[1:]).lower()))
                    except:
                        continue

            vtt1 = vtt2
        file_content_ls.append(vtt1)
    
file_content_dict = dict(zip(file_name_ls,file_content_ls))

In [15]:
writer = ix.writer()
length = len(file_content_dict)
for i, f in enumerate(file_content_dict):
    print(f'{i+1} of {length}', end='\r')
    tmp = ''
    for caption in file_content_dict[f]:
        tmp += caption[1] + ' '
        
    writer.add_document(title=f, path="/"+f, content=tmp)

writer.commit()

In [16]:
from whoosh.qparser import QueryParser

In [17]:
def toseconds(timepoint):
    tmp = timepoint.split('.')[0].split(':')
    hours = int(tmp[0])
    minutes = int(tmp[1])
    seconds = int(tmp[2])
    return str(hours*360 + minutes*60 + seconds)

def searchCorpus(q):
    with ix.searcher() as searcher:
        query = QueryParser('content', ix.schema).parse(q)
        results = searcher.search(query, limit=10)
        results.fragmenter.charlimit = None
        # Show more context before and after
        results.fragmenter.surround = 400
        for hit in results:
            print(hit['title'][:-19] + '\n')

            filecontents = ''
            for line in file_content_dict[hit['title']]:
                filecontents += '\n' + line[0] + '\t' + line[1]
            
            snippets = hit.highlights('content', text=filecontents, top=5).split('...')
            snippets = [cleanhtml(x) for x in snippets]
            for snippet in snippets:
                surrounding_lines = snippet.split('\n')[1:-1]
                for i, line in enumerate(surrounding_lines):
                    tmp = line.split('\t')
                    print('\t' + tmp[3])
                    
                    if(i == 0):
                        video_id = tmp[0][-18:-7]
                        time_start = toseconds(tmp[1])
                
                try:
                    if(len(surrounding_lines) > 0):
                        print('\thttps://chimeraeditor.com/app/editor/?v=' + video_id + '&t=' + time_start + '\n')
                except:
                    continue
                
            
        return results

In [18]:
searchCorpus('algorithms')

Lec 20 _ MIT RES.6-008 Digital Signal Processing, 1975

	all right, that corresponds then to two possible decimation
	in frequency algorithms.
	one in which the input is in normal order,
	https://chimeraeditor.com/app/editor/?v=xRLaQ4My3ms&t=478

	the computation of the dft--
	in other words, the fft algorithms--
	play a role is in implementing convolution or correlation.
	https://chimeraeditor.com/app/editor/?v=xRLaQ4My3ms&t=1589

	however, in this case the coefficients
	are in normal order.
	so if we match up the algorithms that way,
	then we're faced with the problem
	https://chimeraeditor.com/app/editor/?v=xRLaQ4My3ms&t=1731

	of other computational issues to be
	considered in implementing the fast fourier transform
	algorithms.
	some of these are discussed in the text.
	https://chimeraeditor.com/app/editor/?v=xRLaQ4My3ms&t=1878

	terms.
	and we can proceed, essentially, along a root
	similar to the decimation in time algorithms or along a root
	similar to the decimation in frequen

<Top 10 Results for Term('content', 'algorithms') runtime=0.008913728000010224>